In [1]:
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0
! pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#scrape table from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

# drop Not assigned rows
df = df[df.Borough != "Not assigned"]

# reset index
df.reset_index(drop=True, inplace=True)

df.shape

(103, 3)

In [3]:
# upload geodata from csv
df1 = pd.read_csv('http://cocl.us/Geospatial_data')

In [4]:
# add geolocation data to the main table
df2 = pd.merge(df, df1, on='Postal Code', how='outer')
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [6]:
#get Toronto geolocation

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
#prepare data foclustering, remove non numerical columns
toronto_grouped_clustering = df2.drop(['Postal Code','Borough','Neighbourhood'], 1)
toronto_grouped_clustering.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494


In [9]:
# run k-means clustering
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 2, 0, 2, 1, 3, 4, 4, 2], dtype=int32)

In [10]:
# insert Cluster column to the main table
df2.insert(0, 'Cluster Labels', kmeans.labels_)


In [11]:
df2.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
# create map of Toronto with clasters of neigbourhoods
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighbourhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [1]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxx
CLIENT_SECRET:xxx


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
# type your answer here
df3 = getNearbyVenues(names=df2['Neighbourhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [16]:
print(df3.shape)
df3.head()

(2129, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [17]:
df3.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,59,59,59,59,59,59
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [18]:
print('There are {} uniques categories.'.format(len(df3['Venue Category'].unique())))

There are 269 uniques categories.


In [19]:
# one hot encoding
df3_onehot = pd.get_dummies(df3[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df3_onehot['Neighborhood'] = df3['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df3_onehot.columns[-1]] + list(df3_onehot.columns[:-1])
df3_onehot = df3_onehot[fixed_columns]

df3_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
df3_onehot.shape

(2129, 269)

In [21]:
df4_onehot = df3_onehot[['Neighborhood','Athletics & Sports','Bakery','Baseball Field','Beach','Bus Line','Bus Station','Bus Stop','Curling Ice','Department Store','Discount Store','Dog Run','Drugstore','Farmers Market','Field','Fruit & Vegetable Store','Garden','Golf Course','Grocery Store','Harbor / Marina','Health Food Store','Historic Site','Hockey Arena','Hospital','Lake','Light Rail Station','Market','Medical Center','Metro Station','Organic Grocery','Other Great Outdoors','Park','Pharmacy','Playground','River','Scenic Lookout','Skate Park','Skating Rink','Soccer Field','Supermarket','Tennis Court','Trail','Train Station']]

In [22]:
df4_onehot.head()

,Neighborhood,Athletics & Sports,Bakery,Baseball Field,Beach,Bus Line,Bus Station,Bus Stop,Curling Ice,Department Store,Discount Store,Dog Run,Drugstore,Farmers Market,Field,Fruit & Vegetable Store,Garden,Golf Course,Grocery Store,Harbor / Marina,Health Food Store,Historic Site,Hockey Arena,Hospital,Lake,Light Rail Station,Market,Medical Center,Metro Station,Organic Grocery,Other Great Outdoors,Park,Pharmacy,Playground,River,Scenic Lookout,Skate Park,Skating Rink,Soccer Field,Supermarket,Tennis Court,Trail,Train Station
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df4 = df4_onehot.groupby('Neighborhood').mean().reset_index()
df4

,Neighborhood,Athletics & Sports,Bakery,Baseball Field,Beach,Bus Line,Bus Station,Bus Stop,Curling Ice,Department Store,Discount Store,Dog Run,Drugstore,Farmers Market,Field,Fruit & Vegetable Store,Garden,Golf Course,Grocery Store,Harbor / Marina,Health Food Store,Historic Site,Hockey Arena,Hospital,Lake,Light Rail Station,Market,Medical Center,Metro Station,Organic Grocery,Other Great Outdoors,Park,Pharmacy,Playground,River,Scenic Lookout,Skate Park,Skating Rink,Soccer Field,Supermarket,Tennis Court,Trail,Train Station
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.250000,0.0,0.000000,0.000000,0.000000,0.00
1,"Alderwood, Long Branch",0.125000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.043478,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.043478,0.043478,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.043478,0.000000,0.000000,0.00
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.043478,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.043478,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00
5,Berczy Park,0.000000,0.033898,0.000000,0.016949,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.033898,0.00,0.000000,0.0000,0.0,0.016949,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.016949,0.016949,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.250000,0.0,0.000000,0.000000,0.000000,0.00
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.045455,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.045455,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500,0.00,0.000000,0.0625,0.0,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.125000,0.000000,0.000,0.000000,0.000000,0.00,0.062500,0.000000,0.000000,0.000000,0.00,0.0625,0.000000,0.0,0.000000,0.000000,0.000000,0.00
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0000,0.0,0.000000,0.0625,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0

In [24]:
num_top_venues = 5

for hood in df4['Neighborhood']:
    print("----"+hood+"----")
    temp = df4[df4['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0        Skating Rink  0.25
1  Athletics & Sports  0.00
2            Pharmacy  0.00
3                Lake  0.00
4  Light Rail Station  0.00


----Alderwood, Long Branch----
                venue  freq
0  Athletics & Sports  0.12
1            Pharmacy  0.12
2                Lake  0.00
3  Light Rail Station  0.00
4              Market  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0            Pharmacy  0.04
1         Supermarket  0.04
2       Grocery Store  0.04
3                Park  0.04
4  Athletics & Sports  0.00


----Bayview Village----
                venue  freq
0  Athletics & Sports   0.0
1            Pharmacy   0.0
2                Lake   0.0
3  Light Rail Station   0.0
4              Market   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0            Pharmacy  0.04
1       Grocery Store  0.04
2  Athletics & Sports  0.00
3  Light Rail Station  0.00
4        

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df4['Neighborhood']

for ind in np.arange(df4.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df4.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Train Station,Discount Store,Grocery Store,Golf Course,Garden,Fruit & Vegetable Store,Field,Farmers Market,Drugstore
1,"Alderwood, Long Branch",Athletics & Sports,Pharmacy,Bakery,Dog Run,Grocery Store,Golf Course,Garden,Fruit & Vegetable Store,Field,Farmers Market
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Supermarket,Pharmacy,Grocery Store,Baseball Field,Beach,Golf Course,Garden,Fruit & Vegetable Store,Field
3,Bayview Village,Train Station,Dog Run,Harbor / Marina,Grocery Store,Golf Course,Garden,Fruit & Vegetable Store,Field,Farmers Market,Drugstore
4,"Bedford Park, Lawrence Manor East",Pharmacy,Grocery Store,Discount Store,Golf Course,Garden,Fruit & Vegetable Store,Field,Farmers Market,Drugstore,Dog Run


In [27]:
df4.head()

,Neighborhood,Athletics & Sports,Bakery,Baseball Field,Beach,Bus Line,Bus Station,Bus Stop,Curling Ice,Department Store,Discount Store,Dog Run,Drugstore,Farmers Market,Field,Fruit & Vegetable Store,Garden,Golf Course,Grocery Store,Harbor / Marina,Health Food Store,Historic Site,Hockey Arena,Hospital,Lake,Light Rail Station,Market,Medical Center,Metro Station,Organic Grocery,Other Great Outdoors,Park,Pharmacy,Playground,River,Scenic Lookout,Skate Park,Skating Rink,Soccer Field,Supermarket,Tennis Court,Trail,Train Station
0,Agincourt,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.125000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.043478,0.0,0.0,0.0,0.0,0.00,0.0,0.043478,0.0,0.0,0.0
3,Bayview Village,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.043478,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0


In [31]:
df_park = df4[['Neighborhood','Park']]

In [33]:
df_park.nlargest(10,'Park')

,Neighborhood,Park
88,"Willowdale, Newtonbrook",1.000000
10,Caledonia-Fairbanks,0.500000
65,Rosedale,0.500000
93,York Mills West,0.500000
50,"Milliken, Agincourt North, Steeles East, L'Amo...",0.333333
81,"The Kingsway, Montgomery Road, Old Mill North",0.333333
26,"East Toronto, Broadview North (Old East York)",0.250000
30,"Forest Hill North & West, Forest Hill Road Park",0.250000
44,"Kingsview Village, St. Phillips, Martin Grove ...",0.250000
46,Lawrence Park,0.250000


In [35]:
df_dog = df4[['Neighborhood','Dog Run']]
df_dog.nlargest(5,'Dog Run')

,Neighborhood,Dog Run
37,Hillcrest Village,0.200000
59,"Parkdale, Roncesvalles",0.071429
15,Church and Wellesley,0.012500
0,Agincourt,0.000000
1,"Alderwood, Long Branch",0.000000


In [40]:
df_med = df4[['Neighborhood','Hospital','Medical Center','Pharmacy']]
df_med.nlargest(5,'Hospital')

,Neighborhood,Hospital,Medical Center,Pharmacy
43,"Kensington Market, Chinatown, Grange Park",0.015385,0.0,0.015385
0,Agincourt,0.000000,0.0,0.000000
1,"Alderwood, Long Branch",0.000000,0.0,0.125000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.043478
3,Bayview Village,0.000000,0.0,0.000000


In [41]:
df4.corr()

,Athletics & Sports,Bakery,Baseball Field,Beach,Bus Line,Bus Station,Bus Stop,Curling Ice,Department Store,Discount Store,Dog Run,Drugstore,Farmers Market,Field,Fruit & Vegetable Store,Garden,Golf Course,Grocery Store,Harbor / Marina,Health Food Store,Historic Site,Hockey Arena,Hospital,Lake,Light Rail Station,Market,Medical Center,Metro Station,Organic Grocery,Other Great Outdoors,Park,Pharmacy,Playground,River,Scenic Lookout,Skate Park,Skating Rink,Soccer Field,Supermarket,Tennis Court,Trail,Train Station
Athletics & Sports,1.000000,0.036051,-0.008427,-0.025282,-0.051177,-0.036735,0.254850,0.555525,-0.051752,0.009177,-0.033884,-0.025282,-0.063507,-0.025282,-0.025282,-0.031712,-0.025282,0.014019,-0.025282,-0.025282,-0.025282,-0.035947,-0.025282,-0.035947,-0.034655,-0.025282,-0.025282,-0.034536,-0.025282,-0.025282,-0.027483,0.154695,-0.042608,-0.025282,-0.025282,-0.025282,0.144875,-0.025282,-0.006844,-0.025282,-0.058327,-0.044268
Bakery,0.036051,1.000000,-0.061966,0.030629,0.097458,0.164543,-0.057182,-0.041716,-0.069869,-0.063749,-0.055911,-0.041716,-0.001436,-0.041716,0.007916,-0.051497,-0.041716,-0.054319,-0.041716,-0.041716,0.100563,-0.059315,0.023951,-0.013797,-0.057182,0.103796,-0.041716,0.317851,0.023951,-0.020374,-0.029027,0.017015,-0.067302,-0.041716,0.000967,-0.041716,-0.082925,0.385121,0.050553,-0.041716,-0.094280,-0.023219
Baseball Field,-0.008427,-0.061966,1.000000,-0.016380,-0.033159,-0.021867,-0.022453,-0.016380,-0.031688,-0.022162,-0.021954,-0.016380,-0.041147,-0.016380,-0.016380,-0.020547,-0.016380,-0.014733,-0.016380,-0.016380,-0.016380,-0.023291,-0.016380,-0.023291,-0.022453,-0.016380,-0.016380,-0.022376,-0.016380,-0.016380,-0.060727,-0.059087,-0.027606,-0.016380,-0.016380,-0.016380,-0.032957,-0.016380,-0.040927,-0.016380,-0.037791,-0.028682
Beach,-0.025282,0.030629,-0.016380,1.000000,-0.021767,-0.015624,-0.014739,-0.010753,-0.022011,-0.030229,-0.014411,-0.010753,0.358304,-0.010753,-0.010753,-0.013488,-0.010753,-0.005162,-0.010753,-0.010753,-0.010753,-0.015289,-0.010753,-0.015289,-0.014739,-0.010753,-0.010753,-0.014689,-0.010753,-0.010753,-0.034820,0.006748,-0.018122,-0.010753,-0.010753,-0.010753,-0.021634,-0.010753,-0.026866,-0.010753,-0.024807,-0.018828
Bus Line,-0.051177,0.097458,-0.033159,-0.021767,1.000000,0.190643,-0.029836,-0.021767,-0.044557,-0.061193,-0.029173,-0.021767,-0.054677,-0.021767,-0.021767,-0.027303,-0.021767,-0.075565,-0.021767,-0.021767,-0.021767,-0.030949,-0.021767,-0.030949,-0.029836,-0.021767,-0.021767,0.348539,-0.021767,-0.021767,0.121032,-0.079860,-0.036684,-0.021767,-0.021767,-0.021767,-0.043794,0.408983,-0.054385,-0.021767,-0.050218,-0.038113
Bus Station,-0.036735,0.164543,-0.021867,-0.015624,0.190643,1.000000,-0.021417,-0.015624,0.686015,0.418774,-0.020940,-0.015624,-0.039247,-0.015624,-0.015624,-0.019598,-0.015624,-0.054241,-0.015624,-0.015624,-0.015624,-0.022216,-0.015624,-0.022216,-0.021417,-0.015624,-0.015624,0.436676,-0.015624,-0.015624,-0.031786,-0.054190,-0.026332,-0.015624,-0.015624,-0.015624,-0.031435,0.505933,-0.039037,-0.015624,-0.036046,-0.027357
Bus Stop,0.254850,-0.057182,-0.022453,-0.014739,-0.029836,-0.021417,1.000000,0.489074,-0.030172,-0.041437,-0.019754,-0.014739,-0.037025,-0.014739,-0.014739,-0.018488,-0.014739,-0.051169,-0.014739,-0.014739,-0.014739,-0.020957,-0.014739,-0.020957,-0.020204,-0.014739,-0.014739,-0.020134,-0.014739,-0.014739,0.137782,-0.054077,-0.024840,-0.014739,-0.014739,-0.014739,0.140138,-0.014739,-0.036826,-0.014739,-0.034005,-0.025808
Curling Ice,0.555525,-0.041716,-0.016380,-0.010753,-0.021767,-0.015624,0.489074,1.000000,-0.022011,-0.030229,-0.014411,-0.010753,-0.027011,-0.010753,-0.010753,-0.013488,-0.010753,-0.037329,-0.010753,-0.010753,-0.010753,-0.015289,-0.010753,-0.015289,-0.014739,-0.010753,-0.010753,-0.014689,-0.010753,-0.010753,0.052974,-0.039451,-0.018122,-0.010753,-0.010753,-0.010753,0.319005,-0.010753,-0.026866,-0.010753,-0.024807,-0.018828
Department Store,-0.051752,-0.069869,-0.031688,-0.022011,-0.044557,0.686015,-0.030172,-0.

In [42]:
df4.describe()

,Athletics & Sports,Bakery,Baseball Field,Beach,Bus Line,Bus Station,Bus Stop,Curling Ice,Department Store,Discount Store,Dog Run,Drugstore,Farmers Market,Field,Fruit & Vegetable Store,Garden,Golf Course,Grocery Store,Harbor / Marina,Health Food Store,Historic Site,Hockey Arena,Hospital,Lake,Light Rail Station,Market,Medical Center,Metro Station,Organic Grocery,Other Great Outdoors,Park,Pharmacy,Playground,River,Scenic Lookout,Skate Park,Skating Rink,Soccer Field,Supermarket,Tennis Court,Trail,Train Station
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.00000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,0.006218,0.019547,0.009733,0.000180,0.010106,0.002996,0.004179,0.001520,0.004429,0.009364,0.003021,0.002660,0.002376,0.002660,0.000247,0.003431,0.002128,0.019669,0.000665,0.002660,0.000236,0.005319,0.000164,0.000213,0.00209,0.000242,0.001330,0.001655,0.000164,0.000106,0.066885,0.014473,0.019745,0.003546,0.000319,0.000665,0.009073,0.001064,0.003331,0.003546,0.014432,0.000319
std,0.025642,0.048848,0.061946,0.001748,0.048405,0.019988,0.029561,0.014735,0.020979,0.032295,0.021850,0.025786,0.009172,0.025786,0.002399,0.026518,0.020628,0.054931,0.006446,0.025786,0.002292,0.036270,0.001587,0.001451,0.01478,0.002344,0.012893,0.011745,0.001587,0.001031,0.149510,0.038247,0.113591,0.034381,0.003094,0.006446,0.043721,0.010314,0.012926,0.034381,0.060648,0.001767
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.049405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.142857,0.250000,0.500000,0.016949,0.250000,0.166667,0.250000,0.142857,0.166667,0.200000,0.200000,0.250000,0.062500,0.250000,0.023256,0.250000,0.200000,0.266667,0.062500,0.250000,0.022222,0.250000,0.015385,0.010000,0.12500,0.022727,0.125000,0.100000,0.015385,0.010000,1.000000,0.250000,1.000000,0.333333,0.030000,0.062500,0.250000,0.100000,0.071429,0.333333,0.333333,0.010000
